# Climate Protection Actions Around the World

Looking at climate change performance index via https://en.wikipedia.org/wiki/Climate_Change_Performance_Index#2022_results

Goal: Produce interactive map that allows for comparison of climate change index across countries as well as more information about what specifically is contributing to to climate change and what needs to be altered.

In [1]:
import pandas as pd
import plotly.express as px

## Data Importing, Cleaning, and Preparation

### Loading Base Data

Base data contains the 2022 Climate Change Performance Index results from  https://en.wikipedia.org/wiki/Climate_Change_Performance_Index#2022_results

In [2]:
# Remove all rows with NaN values because all information is necessary
ccpi = pd.read_csv("data/ccpi.csv").dropna()
ccpi.head()

,rank,country,iso,score
0,1,Denmark,DNK,76.67
1,2,Sweden,SWE,74.22
2,3,Norway,NOR,73.29
3,4,United Kingdom,GBR,73.09
4,5,Morocco,MAR,71.60


### Importing Emissions Data

Emissions data contains information about different gasseous emissions ranging from 1750 to 2020 and is from https://www.kaggle.com/datasets/danielrpdias/co2-and-greenhouse-gas-emissions

In [18]:
# Drop rows where an ISO Code is not provided as they cannot be used on visualization
emissions = pd.read_csv("data/emissions.csv").dropna(subset=["iso_code"])
emissions.head()

,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,co2_per_capita,consumption_co2_per_capita,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,AFG,Afghanistan,1949,0.015,NaN,NaN,NaN,NaN,0.002,NaN,...,NaN,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1950,0.084,NaN,475.0,0.070,NaN,0.011,NaN,...,NaN,NaN,NaN,NaN,NaN,7752117.0,9.421400e+09,NaN,NaN,NaN
2,AFG,Afghanistan,1951,0.092,NaN,8.7,0.007,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,NaN,7840151.0,9.692280e+09,NaN,NaN,NaN
3,AFG,Afghanistan,1952,0.092,NaN,0.0,0.000,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,NaN,7935996.0,1.001732e+10,NaN,NaN,NaN
4,AFG,Afghanistan,1953,0.106,NaN,16.0,0.015,NaN,0.013,NaN,...,NaN,NaN,NaN,NaN,NaN,8039684.0,1.063052e+10,NaN,NaN,NaN


In [ ]:
# Query only most recent available year of emissions
# Reset index and drop columns where all values are NaN
emissions_last = emissions[emissions["year"] == max(emissions["year"])].reset_index(drop=True).dropna(axis="columns", how="all")
emissions_last.head()

In [19]:
list(emissions_last.columns)

['iso_code',
 'country',
 'year',
 'co2',
 'consumption_co2',
 'co2_growth_prct',
 'co2_growth_abs',
 'trade_co2',
 'co2_per_capita',
 'consumption_co2_per_capita',
 'share_global_co2',
 'cumulative_co2',
 'share_global_cumulative_co2',
 'coal_co2',
 'cement_co2',
 'flaring_co2',
 'gas_co2',
 'oil_co2',
 'other_industry_co2',
 'cement_co2_per_capita',
 'coal_co2_per_capita',
 'flaring_co2_per_capita',
 'gas_co2_per_capita',
 'oil_co2_per_capita',
 'other_co2_per_capita',
 'trade_co2_share',
 'share_global_cement_co2',
 'share_global_coal_co2',
 'share_global_flaring_co2',
 'share_global_gas_co2',
 'share_global_oil_co2',
 'share_global_other_co2',
 'cumulative_cement_co2',
 'cumulative_coal_co2',
 'cumulative_flaring_co2',
 'cumulative_gas_co2',
 'cumulative_oil_co2',
 'cumulative_other_co2',
 'share_global_cumulative_cement_co2',
 'share_global_cumulative_coal_co2',
 'share_global_cumulative_flaring_co2',
 'share_global_cumulative_gas_co2',
 'share_global_cumulative_oil_co2',
 'share_

### Joining Data Sources

In [17]:
ccpi_emissions = ccpi.merge(right=emissions_last.loc[:, emissions_last.columns != "country"], how="inner", left_on="iso", right_on="iso_code").drop("iso_code", axis=1)
ccpi_emissions.dropna(axis="columns", how="all", inplace=True)
ccpi_emissions.head()

,rank,country,iso,score,year,co2,co2_growth_prct,co2_growth_abs,co2_per_capita,share_global_co2,...,cumulative_gas_co2,cumulative_oil_co2,cumulative_other_co2,share_global_cumulative_cement_co2,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,population
0,1,Denmark,DNK,76.67,2020,26.195,-15.22,-4.702,4.522,0.08,...,273.120,1853.778,14.953,0.19,0.23,0.10,0.11,0.31,0.16,5792203.0
1,2,Sweden,SWE,74.22,2020,38.635,-5.62,-2.302,3.826,0.11,...,70.227,3267.419,19.158,0.26,0.19,0.10,0.03,0.55,0.20,10099270.0
2,3,Norway,NOR,73.29,2020,41.283,-3.69,-1.582,7.615,0.12,...,412.919,1405.108,10.503,0.14,0.08,0.61,0.17,0.24,0.11,5421242.0
3,4,United Kingdom,GBR,73.09,2020,329.579,-10.69,-39.431,4.855,0.95,...,7002.510,13418.030,108.013,1.21,7.21,1.64,2.86,2.26,1.15,67886004.0
4,5,Morocco,MAR,71.60,2020,64.536,-5.40,-3.680,1.748,0.19,...,32.371,1089.605,NaN,0.39,0.06,NaN,0.01,0.18,NaN,36910558.0


## Plotting Data

In [21]:
fig = px.choropleth(
    ccpi_emissions, 
    locations="iso", 
    color="co2_per_capita", 
    hover_name="country", 
    hover_data=["iso", "rank", "score", "co2_per_capita"],
    color_continuous_scale=px.colors.sequential.Hot[::-1],
    projection='natural earth')

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()